In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.table import Table
from astropy.io import fits
import astropy.units as u
from scipy.interpolate import interp1d
import astropy
import camb
import astroquery
import healpy
import sympy
import treecorr
import Corrfunc
import math
from astropy.wcs import WCS
import healpy as hp
import scipy

In [ ]:
lower_dec_bound=-22.75
upper_dec_bound=-3.66
RA_bound_leftmost=22.57
RA_bound_rightmost=335.
RA_bound_centre_left=0.
RA_bound_centre_right=360.
ra_range=RA_bound_leftmost+RA_bound_centre_right-RA_bound_rightmost
ra_range

In [ ]:
catalogue=astropy.io.fits.open("../Data/XG_IDR2_4Parts_170-231MHz_comp_rescaled.fits")
catalogue_data=catalogue[1].data
catalogue_columns=catalogue[1].columns
catalogue_csv=pd.read_csv("../Data/XG_IDR2_4Parts_170-231MHz_comp_rescaled.csv")
image=astropy.io.fits.open("../Data/XG_IDR2_4Parts_170-231MHz_rescaled.fits")
image_data=image[0].data
positions=astropy.coordinates.SkyCoord(catalogue_csv.iloc[:,6],catalogue_csv.iloc[:,8],unit=u.deg)
rms=astropy.io.fits.open("../Data/XG_IDR2_4Parts_170-231MHz_rms.fits")
rms_data=rms[0].data

In [ ]:
small_area_sources_dec_range=catalogue_csv[catalogue_csv['dec']<=upper_dec_bound]
small_area_sources_dec_range=small_area_sources_dec_range[small_area_sources_dec_range['dec']>=lower_dec_bound]
small_area_sources_1=small_area_sources_dec_range[small_area_sources_dec_range['ra']>=RA_bound_centre_left]
small_area_sources_1=small_area_sources_1[small_area_sources_1['ra']<=RA_bound_leftmost]
small_area_sources_2=small_area_sources_dec_range[small_area_sources_dec_range['ra']>=RA_bound_rightmost]
small_area_sources_2=small_area_sources_2[small_area_sources_2['ra']<=RA_bound_centre_right]
small_area_sources=pd.concat([small_area_sources_1,small_area_sources_2])
small_area_sources.reset_index(inplace=True,drop=True)
small_area_sources.to_csv('../Data/small_area_sources.csv')

In [ ]:
plt.hist(small_area_sources['local_rms'],bins=50)
plt.xlabel('local rms (Jy/beam)')
plt.ylabel('Number of sources with this local rms value')
plt.title('local rms vs N for small area count calculation')

In [ ]:
def poisson_interval(k: np.ndarray, alpha: float=0.05):
    """
    uses chisquared info to get the poisson interval. Uses scipy.stats
    (imports in function).

    Taken from:
    http://stackoverflow.com/questions/14813530/poisson-confidence-interval-with-numpy
    Based on:
    https://en.wikipedia.org/wiki/Poisson_distribution#Confidence_interval
    """
    from scipy.stats import chi2
    a = alpha
    low, high = (chi2.ppf(a/2., 2*k) / 2., chi2.ppf(1.-a/2., 2.*k + 2.) / 2.)

    # If count is zero, set its lower limit to zero
    low[k==0] = 0.0

    return low, high

def euclid_transform(bc: np.ndarray, area: float, h: np.ndarray, db: np.ndarray):
    
    if not isinstance(area, float):
        area_bin = area(bc)
    else:
        area_bin = area
    
    return 10.**(2.5*np.log10(bc) - np.log10(area_bin*0.000304617) + np.log10(h/db))
    
def source_count(xarr: np.ndarray, nbins: int=100, area: float=16.) -> tuple:
    """Calculate the source counts given a set of measurements.
    
    xarr {np.ndarry} - the flux values in units of Jy
    nbins {int} - the number of flux bins to make
    """
    xmin, xmax = xarr.min(), xarr.max()
    bins = np.logspace(np.log10(xmin*0.9999), np.log10(xmax*1.0001), num=nbins)
    hb = np.histogram(xarr, bins=bins, range=(xmin, xmax))
    h, b = hb
    
    # Bin centers
    bc = (b[1:]+b[:-1])/2.
    # Bin widths
    db = np.diff(b)
    
    yerr_l, yerr_u = poisson_interval(h, alpha=0.16)

    y = euclid_transform(bc, area, h, db)
    yerr_l = euclid_transform(bc, area, yerr_l, db) 
    yerr_u = euclid_transform(bc, area, yerr_u, db) 
    errs = np.array([yerr_l - y, y - yerr_u])
    
    return (y, bc, db, errs)

In [ ]:
# ra_pix = np.abs(image[0].header['CDELT1'])*u.deg
# dec_pix = np.abs(image[0].header['CDELT2'])*u.deg
# pix_area = ra_pix*dec_pix
# wcs_RMS=WCS(rms[0].header)
# wcs_image=WCS(image[0].header)
# lower_left_position=astropy.coordinates.SkyCoord(RA_bound_leftmost,lower_dec_bound,unit=u.deg,frame='icrs')
# upper_right_position=astropy.coordinates.SkyCoord(RA_bound_rightmost,upper_dec_bound,unit=u.deg,frame='icrs')
# image_pixel_low_x,image_pixel_low_y=wcs_image.world_to_pixel(lower_left_position)
# image_pixel_low_x=np.int64(image_pixel_low_x)
# image_pixel_low_y=np.int64(image_pixel_low_y)
# image_pixel_high_x,image_pixel_high_y=wcs_image.world_to_pixel(upper_right_position)
# image_pixel_high_x=np.int64(image_pixel_high_x)
# image_pixel_high_y=np.int64(image_pixel_high_x)
# area=pix_area*(image_pixel_high_y-image_pixel_low_y)*(image_pixel_high_x-image_pixel_low_x)
# area

In [ ]:
total_sky_area=4*np.pi*(u.sr.to(u.deg**2))
ra_fraction=ra_range/360

In [ ]:
upper_dec_bound_radian=upper_dec_bound*u.deg.to(u.rad)
lower_dec_bound_radian=lower_dec_bound*u.deg.to(u.rad)

In [ ]:
area_2=(1/2)*abs(np.sin(upper_dec_bound_radian)-np.sin(lower_dec_bound_radian))*(ra_fraction)*total_sky_area
area_2

In [ ]:
# area_3=ra_fraction*total_sky_area*(-1*np.cos(upper_dec_bound_radian)+np.cos(lower_dec_bound_radian))*(1/2)
# area_3

In [ ]:
# David def.
def area_of_segment(x,y):
    '''
    Function compute area of segment, or zone, on a sphere
    :param x: northern extent of zone (maximum declination) in radians
    :param y: southern extent of zone (minimum declination) in radians
    :return: area of zone (band) around the sphere in steradians
    '''
    a = np.sin(x)
    b = np.sin(y)
    h = a-b
    return h
rarange = np.radians(ra_range)
decrange = np.radians(upper_dec_bound)-np.radians(lower_dec_bound)
dec_max = np.radians(upper_dec_bound)
dec_min = np.radians(lower_dec_bound)
area = area_of_segment(dec_max,dec_min)*(rarange) # area in steradians
area

In [ ]:
# RA_bound_rightmost

In [ ]:
# abs(np.sin(upper_dec_bound)-np.sin(lower_dec_bound))

In [ ]:
GLEAM_X_count_small_area = source_count(catalogue_csv['peak_flux'].values, area=area_2, nbins=50)

In [ ]:
fig, ax = plt.subplots(1,1)
ax.errorbar(GLEAM_X_count_small_area[1], GLEAM_X_count_small_area[0], yerr=GLEAM_X_count_small_area[3], fmt='v', label='GLEAM_X_counts_small_area')
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()
plt.xlabel('S (Jy/beam)')
plt.ylabel('euclidian normalised counts')
plt.title('Euclidian normalised counts GLEAM-X (small area)')
fig.show()

In [ ]:
Hales_7c=pd.read_csv('../Data/Hales_7C.csv')
Itermra_GMRT=pd.read_csv('../Data/Itemra_GRMT.csv')
Ghosh_GMRT=pd.read_csv('../Data/Ghosh_GMRT.csv')
Williams_GMRT=pd.read_csv('../Data/Williams_TRAMISU_GMRT.csv')
Williams_GMRT_key_info=Williams_GMRT.loc[:,['RAJ2000','DEJ2000','Sp','e_Sp']]
Ghosh_GMRT_key_info=Ghosh_GMRT.loc[:,['_RAJ2000','_DEJ2000','Fpeak','Noise']]
Hales_7C_key_info=Hales_7c.loc[:,['_RAJ2000','_DEJ2000','S','SNR']]
Itermra_GMRT_key_info=Itermra_GMRT.loc[:,['RAJ2000','DEJ2000','S153','e_S153']]

In [ ]:
Hales_7C_key_info.rename(columns={'_RAJ2000':'RAJ2000','_DEJ2000':'DEJ2000','S':'Sp','SNR':'e_Sp'},inplace=True)
Williams_GMRT_key_info.rename(columns={'RAJ2000':'RAJ2000','DEJ2000':'DEJ2000','Sp':'Sp','e_Sp':'e_Sp'},inplace=True)
Ghosh_GMRT_key_info.rename(columns={'_RAJ2000':'RAJ2000','_DEJ2000':'DEJ2000','Fpeak':'Sp','Noise':'e_Sp'},inplace=True)
Itermra_GMRT_key_info.rename(columns={'RAJ2000':'RAJ2000','DEJ2000':'DEJ2000','S153':'Sp','e_S153':'e_Sp'},inplace=True)
Mega_table=pd.concat([Hales_7C_key_info,Williams_GMRT_key_info,Ghosh_GMRT_key_info,Itermra_GMRT_key_info],ignore_index=True)
Mega_table
Hales_7C_key_info=Hales_7C_key_info[Hales_7C_key_info['Sp']>=0.1]

In [ ]:
Ghosh_area=(5700**2)*((3.22*1*u.arcsec.to(u.deg))**2)+(5500**2)*((3.32*1*u.arcsec.to(u.deg))**2)+(5500**2)*((3.27*1*u.arcsec.to(u.deg))**2)+(5500**2)*((3.40*1*u.arcsec.to(u.deg))**2)
Hales_area=1.7*u.sr.to(u.deg*u.deg)
Williams_area=30*u.deg*u.deg
Iterma_area=11.3*u.deg*u.deg

In [ ]:
Ghosh_area

In [ ]:
Franzen_counts = source_count(Mega_table['Sp'].values, area=6., nbins=50)
Hales_counts=source_count(Hales_7C_key_info['Sp'],area=Hales_area,nbins=50)
Williams_counts=source_count(Williams_GMRT_key_info['Sp']*10**-3,area=30.,nbins=50)
Ghosh_counts=source_count(Ghosh_GMRT_key_info['Sp']*10**-3,area=6.6,nbins=50)
Itermra_couunts=source_count(Itermra_GMRT_key_info['Sp']*10**-3,area=11.3,nbins=50)

In [ ]:
GLEAM_franzen_data=np.array([
[8.12, 2884,1428],
[5.92, 3431,1186],
[4.15, 4315,763],
[2.96, 4269,610],
[2.21, 3101,510],
[1.75, 3506,460],
[1.40, 4599,424],
[1.11,4211,358],
[0.892, 3157,265],
[0.735,3135,269],
[0.628, 3032,238],
[0.524,2366,168],
[0.437,2342,148],
[0.357,2198,112],
[0.284,1764,81],
[0.223,1597,67],
[0.180,1275,51],
[0.141,1024,37],
[0.112,893,31],
[0.0925,803,29],
[0.0777,632,22],
[0.0652,574,19],
[0.0548,516,15],
[0.0458,407,12],
[0.0390,352,11],
[0.0331,297,8]
])

In [ ]:
sr_deg=u.sr.to(u.deg*u.deg)

In [ ]:
fig, ax = plt.subplots(1,1)
ax.errorbar(GLEAM_X_count_small_area[1], GLEAM_X_count_small_area[0], yerr=GLEAM_X_count_small_area[3], fmt='v', label='GLEAM_X_counts_small_area')
ax.errorbar(Hales_counts[1], Hales_counts[0], yerr=0, fmt='v', label='Hales_counts')
ax.errorbar(Itermra_couunts[1], Itermra_couunts[0], yerr=0, fmt='v', label='Itermra_counts')
ax.errorbar(Ghosh_counts[1], Ghosh_counts[0], yerr=0, fmt='v', label='Ghosh_counts')
ax.errorbar(Williams_counts[1], Williams_counts[0], yerr=0, fmt='v', label='Williams_counts')
ax.errorbar(GLEAM_franzen_data[:,0], GLEAM_franzen_data[:,1], yerr=GLEAM_franzen_data[:,2], fmt='v', label='GLEAM_Franzen_data')
ax.set_xscale('log')
ax.set_yscale('log')
plt.xlabel('S (Jy/beam)')
plt.ylabel('euclidian normalised counts')
plt.title('Euclidian normalised counts GLEAM-X (small area)')
plt.legend()
fig.show()